In [1]:
import numpy as np
import copy
from scipy.ndimage import imread
from scipy import misc
from scipy.ndimage import rotate
import matplotlib.pyplot as plt
import os
import re
import tensorflow as tf

C:\Users\james\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
originPath = os.getcwd()
pattern = re.compile(".png$")
dirList=list(filter(lambda x: os.path.isdir(x) and not re.search(pattern, x) , os.listdir()))

In [3]:
dirList

['.git',
 '.ipynb_checkpoints',
 'images_background',
 'images_background_small1',
 'images_background_small2']

In [4]:
def imageGet(currentpath, dictFile, encode ,endswith, exclude=[]):
    dirList= list(filter(lambda x: os.path.isdir(x) , os.listdir()))
    if len(dirList) < 1:
        listFile = list(filter(lambda x: x.endswith(endswith) , os.listdir()))
        if len(listFile) >0:
            path = os.getcwd().split("\\")[-2:]
            path='-'.join(path)
            FileList = list(map(lambda x: os.path.join(os.getcwd(), x) ,listFile))
            labels=[len(encode) for i in range(len(FileList))]
            encode[path] = len(encode)
            dictFile.append(tuple(zip(labels, FileList)))
        return dictFile
    for i in dirList:
        if i in exclude:
            continue
        changePath = os.path.join(currentpath, i)
        os.chdir(changePath)
        imageGet(changePath, dictFile,encode , endswith)

In [5]:
dirList

['.git',
 '.ipynb_checkpoints',
 'images_background',
 'images_background_small1',
 'images_background_small2']

In [6]:
FilePaths = []
encoder = {}
imageGet(os.getcwd(), FilePaths, encoder, ".png",['.ipynb_checkpoints', 'images_background_small1',
 'images_background_small2'])
classNum = len(encoder)
os.chdir(originPath)

In [7]:
FilePaths = np.array(FilePaths)
FilePathsNp=np.reshape(FilePaths,(FilePaths.shape[0]*FilePaths.shape[1], FilePaths.shape[2]))
# np.random.shuffle(FilePathsNp)
FilePathsNp.shape

(19280, 2)

In [8]:
classNum

964

In [9]:
encoderedLists = FilePathsNp[:,0].astype(int)
imagePaths = FilePathsNp[:,1]

In [10]:
encoderedLists
depth = len(encoderedLists)
encodered =tf.one_hot(encoderedLists, classNum)
print(encodered.shape)

(19280, 964)


In [11]:
photos=[misc.imread(x,flatten=True) for x in imagePaths]

C:\Users\james\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  """Entry point for launching an IPython kernel.


In [12]:
def rotGen (npArray, rotationAngles):
    repeats=len(rotationAngles)
    df=np.repeat(npArray, repeats, axis=0)
    for i in range(df.shape[0]):
        i-df.shape[0]/4

In [13]:
dfphoto = np.array(photos)
dfphoto = np.expand_dims(dfphoto,3)

In [15]:
k=3
channels=1
dim_hidden=64
stride = [1,2,2,1]
dtype = tf.float32
conv_initializer =  tf.contrib.layers.xavier_initializer_conv2d(dtype=dtype)
fc_initializer =  tf.contrib.layers.xavier_initializer(dtype=dtype)
data_tf = tf.convert_to_tensor(dfphoto, np.float32)
data_tf =tf.expand_dims(data_tf,3)
print(data_tf.shape)
# data_tf = tf.reshape(data_tf, (-1, data_tf.shape[0],data_tf.shape[1], data_tf.shape[2]))
# print(data_tf.shape)

(19280, 105, 105, 1, 1)


In [16]:
weights = {}
weights['conv1'] = tf.get_variable('conv1', [k, k, channels, dim_hidden], initializer=conv_initializer, dtype=dtype)
weights['b1'] = tf.Variable(tf.zeros([dim_hidden]))
weights['conv2'] = tf.get_variable('conv2', [k, k, dim_hidden, dim_hidden], initializer=conv_initializer, dtype=dtype)
weights['b2'] = tf.Variable(tf.zeros([dim_hidden]))
# weights['conv3'] = tf.get_variable('conv3', [k, k, dim_hidden, dim_hidden], initializer=conv_initializer, dtype=dtype)
# weights['b3'] = tf.Variable(tf.zeros([dim_hidden]))
# weights['conv4'] = tf.get_variable('conv4', [k, k, dim_hidden, dim_hidden], initializer=conv_initializer, dtype=dtype)
# weights['b4'] = tf.Variable(tf.zeros([dim_hidden]))

In [17]:
def covnNet(input,weight, bais):
    conv_output = tf.nn.conv2d(input, weight, stride, 'SAME') + bais
    return tf.nn.relu(conv_output)

In [18]:
ImageContextHolder = tf.placeholder(tf.float32, [None, data_tf.shape[1], data_tf.shape[2],data_tf.shape[3]])
OneHotContextHolder = tf.placeholder(tf.float32, [None, classNum])

hidden = covnNet(ImageContextHolder, weights['conv1'],weights['b1'] )
hidden = covnNet(hidden, weights['conv2'],weights['b2'] )
# hidden = covnNet(hidden, weights['conv3'],weights['b3'] )
# hidden = covnNet(hidden, weights['conv4'],weights['b4'] )

hidden = tf.contrib.layers.flatten(hidden)

weights['w5'] = tf.Variable(tf.random_normal([tf.cast(hidden.shape[1], tf.int32) , classNum]),dtype=dtype, name='w5')
weights['b5'] = tf.Variable(tf.zeros([classNum]), name='b5')

weights['w6_1'] = tf.Variable(tf.random_normal([classNum,128]),dtype=dtype, name='w6_1')
weights['w6_2'] = tf.Variable(tf.random_normal([classNum,128]),dtype=dtype, name="w6_2")
weights['b6'] = tf.Variable(tf.zeros([128*2]), name='b6')

hidden = tf.matmul(weights['w5'],weights['w6_1'])
hidden = tf.matmul(OneHotContextHolder,weights['w6_2'])
hidden = tf.concat([weights['w6_1'],weights['w6_2']],axis=1)
print(hidden.shape)
hidden = tf.nn.relu(tf.add(hidden, weights['b6'] ))

weights['w7'] = tf.Variable(tf.random_normal([128*2,128]),dtype=dtype, name="w6_2")
weights['b7'] = tf.Variable(tf.zeros([128]), name='b7')
hidden = tf.add(tf.matmul(hidden, weights['w7']),weights['b7'])

representation = tf.reduce_mean(hidden, axis=1)
print(representation.shape)

(964, 256)
(964,)


In [19]:
representation=tf.expand_dims(representation,1)

In [20]:
ImageTargetHolder = tf.placeholder(tf.float32, [None, data_tf.shape[1], data_tf.shape[2],data_tf.shape[3]])
OneHotTargetHolder = tf.placeholder(tf.float32, [None,classNum])

weights['conv1_T'] = tf.get_variable('conv1_T', [k, k, channels, dim_hidden], initializer=conv_initializer, dtype=dtype)
weights['b1_T'] = tf.Variable(tf.zeros([dim_hidden]))
weights['conv2_T'] = tf.get_variable('conv2_T', [k, k, dim_hidden, dim_hidden], initializer=conv_initializer, dtype=dtype)
weights['b2_T'] = tf.Variable(tf.zeros([dim_hidden]))
hidden = covnNet(ImageTargetHolder, weights['conv1_T'],weights['b1_T'] )
hidden = covnNet(hidden, weights['conv2_T'],weights['b2_T'] )


In [21]:
hidden = tf.contrib.layers.flatten(hidden)
weights['w3_T'] = tf.Variable(tf.random_normal([tf.cast(hidden.shape[1], tf.int32) , tf.cast(representation.shape[0],tf.int32)]),dtype=dtype, name='w3_T')
weights['b3_T'] = tf.Variable(tf.zeros([tf.cast(representation.shape[0],tf.int32)]), name='b3_T')
weights['w4_T'] = tf.Variable(tf.random_normal([tf.cast(representation.shape[0],tf.int32) ,128]),dtype=dtype, name='w4_T')
hidden = tf.add(tf.matmul(hidden,weights['w3_T']), weights['b3_T'])
hidden = tf.concat([weights['w4_T'],representation],axis=1)
weights['b4_T'] = tf.Variable(tf.zeros([tf.cast(hidden.shape[1],tf.int32)]), name='b4_T')
hidden = tf.add(hidden, weights['b4_T'])

weights['w5_T'] = tf.Variable(tf.random_normal([tf.cast(hidden.shape[1],tf.int32) , classNum]),dtype=dtype, name='w5_T')
weights['b5_T'] = tf.Variable(tf.zeros([classNum]), name='b5_T')

hidden = tf.add(tf.matmul(hidden,weights['w5_T']), weights['b5_T'])
hidden = tf.reduce_mean(hidden, axis=1)

In [22]:
# hidden=tf.nn.sigmoid(hidden)

In [23]:
hidden = tf.expand_dims(hidden,0)

In [24]:
hidden

<tf.Tensor 'ExpandDims_2:0' shape=(1, 964) dtype=float32>

In [25]:
OneHotTargetHolder

<tf.Tensor 'Placeholder_3:0' shape=(?, 964) dtype=float32>

In [26]:
loss=tf.nn.sigmoid_cross_entropy_with_logits(logits=hidden, labels=OneHotTargetHolder) 

In [27]:
predict=tf.argmax(hidden, axis=1)
print(predict)

Tensor("ArgMax:0", shape=(1,), dtype=int64)


In [28]:
loss= tf.reduce_mean(loss)

In [29]:
optimizer = tf.train.AdamOptimizer(1e-4)
train_step = optimizer.minimize(loss)

In [30]:
encodered.dtype

tf.float32

In [31]:
TRAINING_ITERATIONS= 10000
# imagePaths
# encoderedLists
contextNum = int(depth * 0.5)
contextPic = dfphoto[:contextNum,:,:]
contextlabel = encodered[:contextNum]

targetPic= dfphoto[contextNum:,:,:]
targetlabel = encodered[contextNum:]

In [32]:
targetlabel.shape

TensorShape([Dimension(9640), Dimension(964)])

In [33]:
OneHotContextHolder.shape

TensorShape([Dimension(None), Dimension(964)])

In [34]:
with tf.Session() as sess:
    feed_dict = {ImageContextHolder: contextPic, OneHotContextHolder: contextlabel.eval(),  ImageTargetHolder: targetPic, OneHotTargetHolder: targetlabel.eval()}
    sess.run(tf.global_variables_initializer())
    for it in range(TRAINING_ITERATIONS):
        _,loss_reduce =sess.run([train_step, loss], feed_dict)
        if (it % 1000 == 0):
            print(loss_reduce)
    feed_dict = {ImageContextHolder: contextPic, OneHotContextHolder: contextlabel.eval(),  ImageTargetHolder: targetPic, OneHotTargetHolder: targetlabel.eval()}
    predictions=sess.run([predict], feed_dict)

0.713975
0.04326541
0.012007829
0.00894624
0.008266443
0.007953144
0.0077418527
0.0076169292
0.0075457715
0.0075032758
0.0074782996
()
[array([732], dtype=int64)]


In [35]:
contextPic = dfphoto[:contextNum,:,:]
contextlabel = encodered[:contextNum]
targetPic= dfphoto[0,:,:]
targetlabel = encodered[0]
targetPic=tf.expand_dims(targetPic,0)
targetlabel=tf.expand_dims(targetlabel,0)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    feed_dict = {ImageContextHolder: contextPic, OneHotContextHolder: contextlabel.eval(),  ImageTargetHolder: targetPic.eval(), OneHotTargetHolder: targetlabel.eval()}
    predictions=sess.run([hidden], feed_dict)
    print(predictions)
    print(encodered[0].eval())
    print(encoderedLists[0])

[array([[-0.14258292, -0.01135773, -0.25911862,  0.09460506, -0.44374433,
         0.09245823,  0.25431526, -0.3928016 , -0.48691395,  0.5627697 ,
         0.19481817, -0.18132351, -0.21652375,  0.23718779, -0.2997377 ,
         0.04079618, -0.5040477 ,  0.11497135, -0.03864407, -0.17922863,
        -0.11554082,  0.13532312, -0.40023142,  0.09149925,  0.36344704,
        -0.11107116, -0.39824915, -0.17777711,  0.01672463, -0.02603862,
        -0.15771547, -0.32625896, -0.31676137, -0.11084493, -0.43941388,
        -0.4001232 , -0.62928534, -0.09772313, -0.4657404 , -0.2800735 ,
         0.06449645, -0.04879251,  0.386899  , -0.05884011, -0.09590133,
         0.16744176,  0.05613739, -0.0983773 , -0.55905634,  0.279277  ,
        -0.02948554,  0.00359707, -0.32495117, -0.12788187, -0.9031786 ,
        -0.11602707, -0.30447558, -0.12473563,  0.21251023,  0.16242258,
         0.3422519 , -0.21752441, -0.31386822,  0.10047138, -0.24262519,
        -0.59574896, -0.05190565, -0.4338803 , -0.

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [36]:
predictions[0][0][257]

0.1704916